Imports

In [33]:
import mediapipe as mp
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler    
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

# Load DATA

In [42]:
# The data labels are in the form below:
alphabet = ['1', '2', '3','4', 'space', 'del', 
'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
labels_ = alphabet
NUM_CLASSES = len(labels_)

In [35]:
x_flat = pd.read_csv("./data/train/x_data900.csv") 
y_flat = pd.read_csv("./data/train/y_data900.csv")  #Note the first column of this is the index, so it's nx2 size

x_flat_test = pd.read_csv("./data/test/x_data_test100.csv")
y_flat_test = pd.read_csv("./data/test/y_data_test100.csv") #Note the first column of this is the index, so it's nx2 size



In [36]:
# We need to unflatten
def unflatten(df,shape):
    """ 
    reshape a dataframe to the correct array size
    shape is an array ex [21,3] which describes the desired shape
    """
    old_array = df.to_numpy()
    new_array = []
    for point in old_array:
        point_reshaped = point.reshape(shape[0], shape[1])
        new_array.append(point_reshaped)

    return np.array(new_array)

# TEST
arr = np.array([[1,2,3,4,5,6,7,8],[1,2,3,4,5,6,7,8],[1,2,3,4,5,6,7,8]])
arr = pd.DataFrame(arr)
unflatten(arr,shape=[2,4])



array([[[1, 2, 3, 4],
        [5, 6, 7, 8]],

       [[1, 2, 3, 4],
        [5, 6, 7, 8]],

       [[1, 2, 3, 4],
        [5, 6, 7, 8]]])

In [37]:
#revert the data back into its original form
x_data = unflatten(x_flat,shape=[21,3])
X_test = unflatten(x_flat_test,shape=[21,3])

#Adjust labels to numpy array
y_data = y_flat.to_numpy()
y_test = y_flat_test.to_numpy()

print("training data shape:", np.shape(x_data), "training score's shape", np.shape(y_data))
print("testing data shape:", np.shape(X_test), "testing score's shape", np.shape(y_test))


training data shape: (24367, 21, 3) training score's shape (24367, 1)
testing data shape: (2659, 21, 3) testing score's shape (2659, 1)


In [38]:
# Validation split
X_train, X_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.1)

In [39]:
#One hot encode
y_train = to_categorical(y_train).astype(int)
y_valid = to_categorical(y_valid).astype(int)
y_test = to_categorical(y_test).astype(int)

Now we can train our models. 

# Modeling
Our inputs are:
- X_Train, y_train 
- X_valid, y_valid
- X_test,y_test
Each datapoint in the Xs are [21,3]

In [40]:
# Setup checkpoint
import os
import tensorflow as tf

checkpoint_path = "./checkpoints.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)



In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# define the model architecture
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(21, 3)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=NUM_CLASSES, activation='softmax'))

#Compile
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [51]:
model.fit(X_train, y_train, epochs=5, callbacks=[cp_callback],validation_data=(X_valid, y_valid))

Epoch 1/5
673/686 [============================>.] - ETA: 0s - loss: 0.1868 - categorical_accuracy: 0.9737
Epoch 1: saving model to .\checkpoints.ckpt
686/686 [==============================] - 2s 3ms/step - loss: 0.1852 - categorical_accuracy: 0.9739 - val_loss: 0.1521 - val_categorical_accuracy: 0.9713
Epoch 2/5
672/686 [============================>.] - ETA: 0s - loss: 0.1249 - categorical_accuracy: 0.9810
Epoch 2: saving model to .\checkpoints.ckpt
686/686 [==============================] - 1s 2ms/step - loss: 0.1259 - categorical_accuracy: 0.9809 - val_loss: 0.1070 - val_categorical_accuracy: 0.9852
Epoch 3/5
677/686 [============================>.] - ETA: 0s - loss: 0.0960 - categorical_accuracy: 0.9854
Epoch 3: saving model to .\checkpoints.ckpt
686/686 [==============================] - 1s 2ms/step - loss: 0.0975 - categorical_accuracy: 0.9853 - val_loss: 0.0862 - val_categorical_accuracy: 0.9869
Epoch 4/5
665/686 [============================>.] - ETA: 0s - loss: 0.0808 - cate